In [ ]:
import os

In [ ]:
%pwd

In [ ]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [ ]:
%pwd

In [ ]:
with open('.env') as f:
    os.environ.update(
        line.strip().split('=') for line in f
)

### Training Config

This code will be apply in `src/MarketplaceReviews/entity/config_entity.py`.

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    input_train_path: Path
    output_train_path: Path
    vectorized_train_path: Path
    model_path: Path
    params_max_iter: int
    params_solver: str
    params_n_jobs: int

### Training Config Manager

This code will be apply in `src/MarketplaceReviews/config/configurations.py`.

In [ ]:
from MarketplaceReviews.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from MarketplaceReviews.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        """read training config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: TrainingConfig type
        """
        data_dump_config = self.config.dump_data
        vectorize_config = self.config.vectorize_data
        train_config = self.config.train_model
        train_params = self.params

        create_directories([train_config.root_dir])

        config = TrainingConfig(
            root_dir=train_config.root_dir,
            input_train_path=Path(data_dump_config.input_train_path),
            output_train_path=Path(data_dump_config.output_train_path),
            vectorized_train_path=Path(vectorize_config.vectorized_train_path),
            model_path=Path(train_config.model_path),
            params_max_iter=train_params.MAX_ITER,
            params_solver=train_params.SOLVER,
            params_n_jobs=train_params.N_JOBS
        )

        return config

### Perform Training

This code in `src/MarketplaceReviews/components/training.py`.

For this example, for initial run we could use logistic regression, later on we could try:
+ another model
+ another data enrichment technique
+ another model tweaking

In [ ]:
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from MarketplaceReviews import logger

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def logistic_regression(self) -> None:
        """train the data with linear regression model and dump the data
        """
        logger.info(f"Load vectorized data train from {self.config.vectorized_train_path}.")
        X_train_vec = joblib.load(self.config.vectorized_train_path)
        
        logger.info(f"Load data train output from {self.config.output_train_path}.")
        y_train = joblib.load(self.config.output_train_path)
        
        logger.info(f"Train the model.")
        model = LogisticRegression(
            solver=self.config.params_solver,
            max_iter=self.config.params_max_iter,
            n_jobs=self.config.params_n_jobs
        )
        
        model.fit(X_train_vec, y_train)
        
        logger.info(f"Dump the model.")
        joblib.dump(model, self.config.model_path)

### Traning the Model

This code in `src/MarketplaceReviews/pipeline/step_03_training.py`.

In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.logistic_regression()
except Exception as e:
    logger.error(e)
    raise e

**Debug**: Predict by showing the data training prediction result.

In [ ]:
import pandas as pd

X_train = joblib.load(training_config.input_train_path)
X_train_vec = joblib.load(training_config.vectorized_train_path)
y_train = joblib.load(training_config.output_train_path)
model = joblib.load(training_config.model_path)

y_pred = pd.Series(model.predict(X_train_vec), index = X_train.index)
y_pred

In [ ]:
print(f"{classification_report(y_train, y_pred)}")